# GameTheory 19 - Jeux Combinatoires en Lean

**Kernel** : Lean 4 (WSL)

**Navigation** : [<- GameTheory-18-Lean-NashExistence](GameTheory-18-Lean-NashExistence.ipynb) | [Index](GameTheory-1-Setup.ipynb) | [GameTheory-20-Lean-SocialChoice ->](GameTheory-20-Lean-SocialChoice.ipynb)

**Notebook Python compagnon** : [GameTheory-19b-CombinatorialGames-Python](GameTheory-19b-CombinatorialGames-Python.ipynb) (implementations algorithmiques)

---

## Introduction

Ce notebook explore les **jeux combinatoires** formalises dans **mathlib4**. Contrairement aux jeux strategiques (Nash, strategies mixtes), les jeux combinatoires sont des jeux a information parfaite ou deux joueurs jouent alternativement jusqu'a ce qu'un joueur ne puisse plus jouer.

### Jeux strategiques vs Jeux combinatoires

| Aspect | Jeux strategiques | Jeux combinatoires |
|--------|-------------------|--------------------|
| **Joueurs** | N joueurs simultanement | 2 joueurs alternativement |
| **Information** | Peut etre incomplete | Parfaite |
| **Hasard** | Possible | Aucun |
| **Fin** | Gains continus | Un joueur gagne/perd |
| **Exemples** | Poker, Auctions | Echecs, Go, Nim |
| **Theorie** | Nash, mecanismes | Conway, Sprague-Grundy |

### Objectifs

1. Comprendre la structure inductive `PGame` de mathlib
2. Explorer le jeu de Nim et sa valeur de Grundy
3. Decouvrir le theoreme de Sprague-Grundy
4. Introduction aux nombres surreels

### Duree estimee : 50 minutes

---

## 1. Theorie des Jeux Combinatoires

Un **jeu combinatoire** (au sens de Conway) est defini par :
- Deux joueurs : **Gauche (L)** et **Droite (R)**
- Un ensemble de **positions** legales
- Des **regles de deplacement** pour chaque joueur
- Une **condition de fin** : le joueur qui ne peut plus jouer perd

### Classification

| Type | Description | Exemple |
|------|-------------|--------|
| **Impartial** | Memes coups pour les deux joueurs | Nim |
| **Partizan** | Coups differents selon le joueur | Echecs, Domineering |

---

## 2. PGame dans Lean

La definition inductive d'un Pre-Game :

In [1]:
-- Definition simplifiee de PGame
-- (mathlib utilise une version plus elaboree avec univers)

inductive SimplePGame where
  | mk : (L : Type) -> (R : Type) -> (L -> SimplePGame) -> (R -> SimplePGame) -> SimplePGame

-- L = type des coups de Gauche
-- R = type des coups de Droite
-- moveL : L -> SimplePGame = position apres coup de Gauche
-- moveR : R -> SimplePGame = position apres coup de Droite

#check SimplePGame

-- Definition simplifiee de PGame
-- (mathlib utilise une version plus elaboree avec univers)

inductive SimplePGame where
  | mk : (L : Type) -> (R : Type) -> (L -> SimplePGame) -> (R -> SimplePGame) -> SimplePGame

-- L = type des coups de Gauche
-- R = type des coups de Droite
-- moveL : L -> SimplePGame = position apres coup de Gauche
-- moveR : R -> SimplePGame = position apres coup de Droite

#check SimplePGame
──────▶  SimplePGame : Type 1
--% env 0

Raw input:
{"cmd": "-- Definition simplifiee de PGame\n-- (mathlib utilise une version plus elaboree avec univers)\n\ninductive SimplePGame where\n  | mk : (L : Type) -> (R : Type) -> (L -> SimplePGame) -> (R -> SimplePGame) -> SimplePGame\n\n-- L = type des coups de Gauche\n-- R = type des coups de Droite\n-- moveL : L -> SimplePGame = position apres coup de Gauche\n-- moveR : R -> SimplePGame = position apres coup de Droite\n\n#check SimplePGame"}
Raw output:
{"messages":
 [{"severity": "info",
   "pos": {"line": 12, "column": 0},
   "endPos": {"line": 12, "column": 6},
   "data": "SimplePGame : Type 1"}],
 "env": 0}

### Jeux primitifs

In [2]:
-- Jeux fondamentaux

inductive PG where
  | mk : (L R : Type) -> (L -> PG) -> (R -> PG) -> PG

-- 0 : aucun coup pour personne
def PG.zero : PG := PG.mk Empty Empty (fun e => e.elim) (fun e => e.elim)

-- 1 : Gauche peut bouger vers 0, Droite ne peut pas
def PG.one : PG := PG.mk Unit Empty (fun _ => PG.zero) (fun e => e.elim)

-- -1 : Droite peut bouger vers 0, Gauche ne peut pas
def PG.negOne : PG := PG.mk Empty Unit (fun e => e.elim) (fun _ => PG.zero)

-- * (star) : les deux peuvent bouger vers 0
def PG.star : PG := PG.mk Unit Unit (fun _ => PG.zero) (fun _ => PG.zero)

#check PG.zero
#check PG.one
#check PG.negOne
#check PG.star

-- Jeux fondamentaux

inductive PG where
  | mk : (L R : Type) -> (L -> PG) -> (R -> PG) -> PG

-- 0 : aucun coup pour personne
def PG.zero : PG := PG.mk Empty Empty (fun e => e.elim) (fun e => e.elim)

-- 1 : Gauche peut bouger vers 0, Droite ne peut pas
def PG.one : PG := PG.mk Unit Empty (fun _ => PG.zero) (fun e => e.elim)

-- -1 : Droite peut bouger vers 0, Gauche ne peut pas
def PG.negOne : PG := PG.mk Empty Unit (fun e => e.elim) (fun _ => PG.zero)

-- * (star) : les deux peuvent bouger vers 0
def PG.star : PG := PG.mk Unit Unit (fun _ => PG.zero) (fun _ => PG.zero)

#check PG.zero
──────▶  PG.zero : PG
#check PG.one
──────▶  PG.one : PG
#check PG.negOne
──────▶  PG.negOne : PG
#check PG.star
──────▶  PG.star : PG
--% env 1

Raw input:
{"cmd": "-- Jeux fondamentaux\n\ninductive PG where\n  | mk : (L R : Type) -> (L -> PG) -> (R -> PG) -> PG\n\n-- 0 : aucun coup pour personne\ndef PG.zero : PG := PG.mk Empty Empty (fun e => e.elim) (fun e => e.elim)\n\n-- 1 : Gauche peut bouger vers 0, Droite ne peut pas\ndef PG.one : PG := PG.mk Unit Empty (fun _ => PG.zero) (fun e => e.elim)\n\n-- -1 : Droite peut bouger vers 0, Gauche ne peut pas\ndef PG.negOne : PG := PG.mk Empty Unit (fun e => e.elim) (fun _ => PG.zero)\n\n-- * (star) : les deux peuvent bouger vers 0\ndef PG.star : PG := PG.mk Unit Unit (fun _ => PG.zero) (fun _ => PG.zero)\n\n#check PG.zero\n#check PG.one\n#check PG.negOne\n#check PG.star", "env": 0}
Raw output:
{"messages":
 [{"severity": "info",
   "pos": {"line": 18, "column": 0},
   "endPos": {"line": 18, "column": 6},
   "data": "PG.zero : PG"},
  {"severity": "info",
   "pos": {"line": 19, "column": 0},
   "endPos": {"line": 19, "column": 6},
   "data": "PG.one : PG"},
  {"severity": "info",
   "pos": {"line": 20, "column": 0},
   "endPos": {"line": 20, "column": 6},
   "data": "PG.negOne : PG"},
  {"severity": "info",
   "pos": {"line": 21, "column": 0},
   "endPos": {"line": 21, "column": 6},
   "data": "PG.star : PG"}],
 "env": 1}

### Interpretation des jeux primitifs

| Jeu | Notation | Qui gagne ? |
|-----|----------|-------------|
| $0$ | `{|}` | Second joueur (P-position) |
| $1$ | `{0|}` | Gauche (meme en second) |
| $-1$ | `{|0}` | Droite (meme en second) |
| $*$ | `{0|0}` | Premier joueur (N-position) |

---

## 3. Le jeu de Nim

In [3]:
-- Nim avec un tas de n objets
-- nim(0) = 0 (pas de coup)
-- nim(n) = {nim(0), ..., nim(n-1) | nim(0), ..., nim(n-1)}

def nim : Nat -> PG
  | 0 => PG.mk Empty Empty (fun e => e.elim) (fun e => e.elim)
  | n + 1 => PG.mk (Fin (n + 1)) (Fin (n + 1))
      (fun k => nim k.val)   -- Gauche peut aller vers nim(0), ..., nim(n)
      (fun k => nim k.val)   -- Droite aussi (jeu impartial)

#check nim
#check nim 0  -- = 0
#check nim 1  -- = *
#check nim 3

-- Nim avec un tas de n objets
-- nim(0) = 0 (pas de coup)
-- nim(n) = {nim(0), ..., nim(n-1) | nim(0), ..., nim(n-1)}

def nim : Nat -> PG
  | 0 => PG.mk Empty Empty (fun e => e.elim) (fun e => e.elim)
  | n + 1 => PG.mk (Fin (n + 1)) (Fin (n + 1))
      (fun k => nim k.val)   -- Gauche peut aller vers nim(0), ..., nim(n)
      (fun k => nim k.val)   -- Droite aussi (jeu impartial)

#check nim
──────▶  nim : Nat → PG
#check nim 0  -- = 0
──────▶  nim 0 : PG
#check nim 1  -- = *
──────▶  nim 1 : PG
#check nim 3
──────▶  nim 3 : PG
--% env 2

Raw input:
{"cmd": "-- Nim avec un tas de n objets\n-- nim(0) = 0 (pas de coup)\n-- nim(n) = {nim(0), ..., nim(n-1) | nim(0), ..., nim(n-1)}\n\ndef nim : Nat -> PG\n  | 0 => PG.mk Empty Empty (fun e => e.elim) (fun e => e.elim)\n  | n + 1 => PG.mk (Fin (n + 1)) (Fin (n + 1))\n      (fun k => nim k.val)   -- Gauche peut aller vers nim(0), ..., nim(n)\n      (fun k => nim k.val)   -- Droite aussi (jeu impartial)\n\n#check nim\n#check nim 0  -- = 0\n#check nim 1  -- = *\n#check nim 3", "env": 1}
Raw output:
{"messages":
 [{"severity": "info",
   "pos": {"line": 11, "column": 0},
   "endPos": {"line": 11, "column": 6},
   "data": "nim : Nat → PG"},
  {"severity": "info",
   "pos": {"line": 12, "column": 0},
   "endPos": {"line": 12, "column": 6},
   "data": "nim 0 : PG"},
  {"severity": "info",
   "pos": {"line": 13, "column": 0},
   "endPos": {"line": 13, "column": 6},
   "data": "nim 1 : PG"},
  {"severity": "info",
   "pos": {"line": 14, "column": 0},
   "endPos": {"line": 14, "column": 6},
   "data": "nim 3 : PG"}],
 "env": 2}

### Valeur de Grundy

La **valeur de Grundy** d'un jeu impartial $G$ :

$$\text{grundy}(G) = \text{mex}\{\text{grundy}(G') : G' \text{ accessible depuis } G\}$$

### Theoreme de Sprague-Grundy

**Theoreme** : Tout jeu impartial est equivalent a un jeu de Nim.

$$G \equiv \text{nim}(\text{grundy}(G))$$

Pour une somme de jeux :
$$\text{grundy}(G_1 + G_2 + ... + G_n) = \text{grundy}(G_1) \oplus ... \oplus \text{grundy}(G_n)$$

ou $\oplus$ est le XOR binaire.

---

## 4. Exercices

### Exercice 1 : Construire des jeux

Construire :
1. Le jeu $2 = \{1|\}$
2. Le jeu $1/2 = \{0|1\}$
3. Le jeu $\uparrow = \{0|*\}$

In [4]:
-- Solution Exercice 1

-- 1. Le jeu 2 = {1|}
def PG.two : PG := PG.mk Unit Empty (fun _ => PG.one) (fun e => e.elim)

-- 2. Le jeu 1/2 = {0|1}
def PG.half : PG := PG.mk Unit Unit (fun _ => PG.zero) (fun _ => PG.one)

-- 3. Le jeu up = {0|*}
def PG.up : PG := PG.mk Unit Unit (fun _ => PG.zero) (fun _ => PG.star)

#check PG.two
#check PG.half
#check PG.up

-- Solution Exercice 1

-- 1. Le jeu 2 = {1|}
def PG.two : PG := PG.mk Unit Empty (fun _ => PG.one) (fun e => e.elim)

-- 2. Le jeu 1/2 = {0|1}
def PG.half : PG := PG.mk Unit Unit (fun _ => PG.zero) (fun _ => PG.one)

-- 3. Le jeu up = {0|*}
def PG.up : PG := PG.mk Unit Unit (fun _ => PG.zero) (fun _ => PG.star)

#check PG.two
──────▶  PG.two : PG
#check PG.half
──────▶  PG.half : PG
#check PG.up
──────▶  PG.up : PG
--% env 3

Raw input:
{"cmd": "-- Solution Exercice 1\n\n-- 1. Le jeu 2 = {1|}\ndef PG.two : PG := PG.mk Unit Empty (fun _ => PG.one) (fun e => e.elim)\n\n-- 2. Le jeu 1/2 = {0|1}\ndef PG.half : PG := PG.mk Unit Unit (fun _ => PG.zero) (fun _ => PG.one)\n\n-- 3. Le jeu up = {0|*}\ndef PG.up : PG := PG.mk Unit Unit (fun _ => PG.zero) (fun _ => PG.star)\n\n#check PG.two\n#check PG.half\n#check PG.up", "env": 2}
Raw output:
{"messages":
 [{"severity": "info",
   "pos": {"line": 12, "column": 0},
   "endPos": {"line": 12, "column": 6},
   "data": "PG.two : PG"},
  {"severity": "info",
   "pos": {"line": 13, "column": 0},
   "endPos": {"line": 13, "column": 6},
   "data": "PG.half : PG"},
  {"severity": "info",
   "pos": {"line": 14, "column": 0},
   "endPos": {"line": 14, "column": 6},
   "data": "PG.up : PG"}],
 "env": 3}

---

## 5. Nombres Surreels

Les **nombres surreels** sont une extension des reels decouverte par Conway.

Un nombre surreal est un jeu ou les deux joueurs ont les memes options.

| Jeu | Valeur | Construction |
|-----|--------|-------------|
| $\{|\}$ | $0$ | Aucune option |
| $\{0|\}$ | $1$ | Gauche peut aller a 0 |
| $\{|0\}$ | $-1$ | Droite peut aller a 0 |
| $\{0|1\}$ | $1/2$ | Entre 0 et 1 |
| $\{0,1|\}$ | $2$ | Gauche peut aller a 0 ou 1 |

Dans mathlib4 : `Mathlib.SetTheory.Surreal.Basic`

---

## 6. API mathlib4

### Modules principaux

| Module | Contenu |
|--------|--------|
| `Mathlib.SetTheory.PGame.Basic` | Definition PGame, relations |
| `Mathlib.SetTheory.Game.Nim` | nim, grundyValue, Sprague-Grundy |
| `Mathlib.SetTheory.Surreal.Basic` | Nombres surreels |

### Principales definitions

```lean
-- PGame
inductive PGame
  | mk : (a b : Type u) -> (a -> PGame) -> (b -> PGame) -> PGame

-- Accesseurs
def LeftMoves (G : PGame) : Type
def RightMoves (G : PGame) : Type
def moveLeft (G : PGame) : G.LeftMoves -> PGame
def moveRight (G : PGame) : G.RightMoves -> PGame

-- Nim et Grundy
def nim (n : Ordinal) : PGame
def grundyValue (G : Game) : Ordinal

-- Theoreme Sprague-Grundy
theorem equiv_nim_grundyValue (G : Game) [G.Impartial] :
    G ~ nim (grundyValue G)
```

---

## Resume

| Concept | Definition | Exemple |
|---------|------------|--------|
| **PGame** | Jeu defini inductivement par coups L/R | $\{0|\} = 1$ |
| **Nim** | Jeu impartial avec tas | nim(3) = \{0,1,2 | 0,1,2\} |
| **Grundy** | mex des valeurs des successeurs | grundy(nim(n)) = n |
| **Sprague-Grundy** | Tout jeu impartial ~ nim(grundy) | |
| **Surreal** | Jeu numerique quotiente | 1/2 = \{0|1\} |

### Pour aller plus loin

- **Notebook 19b** : Implementations Python (algorithmes Grundy, Nim)
- **Notebook 20** : Choix social (Arrow, Sen, electeur median)

---

**Navigation** : [<- GameTheory-18-Lean-NashExistence](GameTheory-18-Lean-NashExistence.ipynb) | [Index](GameTheory-1-Setup.ipynb) | [GameTheory-20-Lean-SocialChoice ->](GameTheory-20-Lean-SocialChoice.ipynb)